In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
data = open("dataset_NB.txt", "r")

In [3]:
#returns dataframe containing the data from the dataset provided
def dframe():
    arraydata = [[]]
    for line in data:
        string = str(line)
        result = int(line[-2:])
        if (result == 0):
          arraydata.append([string[:-3],0])
        else:
         arraydata.append([string[:-3],1])
    data.close()
    df = pd.DataFrame(arraydata, columns = ['Statement', 'Sentiment'])
    df = df.dropna()
    return df
 

In [4]:
dataframe = dframe()
dataframe = dataframe.sample(frac=1, random_state=2)
sevendframe = []
training_data = []
testing_data = []

length = round(len(dataframe) * 1/7)

for i in range(0,6,1):
  sevendframe.append(  dataframe[i*length:(i+1)*length].reset_index(drop=True))
sevendframe.append(dataframe[6*length:].reset_index(drop=True))

for j in range(0,7):
  testing_data.append(sevendframe[j])
  train_data = []
  for i in range(0,7): 
    if i!=j: 
      train_data.append(sevendframe[i])
  training_data.append( pd.concat(train_data))



In [5]:
import re

In [6]:
def classify_test_set(message):
   
   p_spam_message = p_spam
   p_ham_message = p_ham
   message = re.sub('\W', ' ', message) #message is the string parsed
   message = message.lower().split() 

  

   for word in message:
      if word in parameters_spam:
         p_spam_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_message *= parameters_ham[word]

   if p_ham_message > p_spam_message:
      return 1
   elif p_spam_message > p_ham_message:
      return 0
   else:
      return 'cannot classify'

In [7]:
sumt=0

for x in range(7):
    training_set = training_data[x]
    test_set = testing_data[x]
    #removing punctuations from the training set
    training_set['Statement'] = training_set['Statement'].str.lower()
    training_set['Statement'] = training_set['Statement'].str.replace('the', ' ')
    training_set['Statement'] = training_set['Statement'].str.replace(',', ' ')
    training_set['Statement'] = training_set['Statement'].str.replace( '.', ' ')
    training_set['Statement'] = training_set['Statement'].str.replace( '(', ' ')
    training_set['Statement'] = training_set['Statement'].str.replace(')', ' ')
    training_set['Statement'] = training_set['Statement'].str.replace('-', ' ')
    training_set['Statement'] = training_set['Statement'].str.replace('!', ' ')
    training_set['Statement'] = training_set['Statement'].str.replace('\d+', ' ')
    training_set['Statement'] = training_set['Statement'].str.split()

    vocab = []
    n_vocabulary = len(vocab)
    
    for statement in training_set['Statement']:
        for word in statement:
          vocab.append(word)

    vocab = list(set(vocab))
    wcps = {unique_word: [0] * len(training_set['Statement']) for unique_word in vocab}

    for index, sms in enumerate(training_set['Statement']):
        for word in sms:
          wcps[word][index] += 1
        
        
    word_counts = pd.DataFrame(wcps)
    word_counts.reset_index(drop=True, inplace=True)
    training_set.reset_index(drop=True, inplace=True)
    
    
    training_set_clean = pd.concat([training_set, word_counts], axis=1)
    spam_messages = training_set_clean[training_set_clean['Sentiment'] == 0]
    ham_messages = training_set_clean[training_set_clean['Sentiment'] == 1]


    p_spam = len(spam_messages) / len(training_set_clean) #probabilty of spam
    p_ham = len(ham_messages) / len(training_set_clean)   #probability of not spam

    noofwordsperspam = spam_messages['Statement'].apply(len)
    n_spam = noofwordsperspam.sum()
    

    noofwordsperham = ham_messages['Statement'].apply(len)
    n_ham = noofwordsperham.sum()


    


    laplacesmoothener = 1    #laplace smoothing
    parameters_spam = {unique_word:0 for unique_word in vocab}
    parameters_ham = {unique_word:0 for unique_word in vocab}

#calculating parameters
    for word in vocab:
       noofwords_spam = spam_messages[word].sum() 
       p_word_given_spam = (noofwords_spam + laplacesmoothener) / (n_spam + laplacesmoothener*n_vocabulary)
       parameters_spam[word] = p_word_given_spam

       noofwords_ham = ham_messages[word].sum() 
       p_word_given_ham = (noofwords_ham + laplacesmoothener) / (n_ham + laplacesmoothener*n_vocabulary)
       parameters_ham[word] = p_word_given_ham
        
        
    test_set['predicted'] = test_set['Statement'].apply(classify_test_set)
    matched = 0
    total = test_set.shape[0]

    for row in test_set.iterrows():
       row = row[1]
       if row['Sentiment'] == row['predicted']:
          matched += 1
    print('Accuracy:', matched*100/total)
    sumt=sumt+(matched*100/total)
print('Average accuracy:', sumt/7)    


Accuracy: 84.61538461538461
Accuracy: 81.81818181818181
Accuracy: 82.51748251748252
Accuracy: 84.61538461538461
Accuracy: 83.91608391608392
Accuracy: 75.52447552447552
Accuracy: 83.80281690140845
Average accuracy: 82.40140141548594
